# Setup
Imports, transducer type, array gemetry, local function definitions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize

from ipywidgets import IntProgress
import multiprocessing
import itertools

import levitate

def dB(val): return 20*np.log10(np.abs(val))
def SPL(val): return dB(val/20e-6)
def wrap_phase(phase): return np.mod(phase + np.pi, 2*np.pi) - np.pi
def normalized(v, axis=None): return v / np.sum(v**2, axis=axis)**0.5

In [ ]:
transducer = levitate.models.CircularRing(freq=40e3, effective_radius=2.9e-3)
# transducer = levitate.models.ReflectingTransducer(
#     levitate.models.CircularRing, effective_radius=2.9e-3, freq=40e3,
#     plane_distance=0, plane_normal=(0,0,1), reflection_coefficient=-1)

array_height = 10 * transducer.wavelength
shape = (16, 16)
t_size = 10e-3
# grid = levitate.models.rectangular_grid(shape=shape, spread=t_size, offset=(0,0,-array_height/2))
grid = levitate.models.double_sided_grid(separation=array_height, offset=(0,0,-array_height/2), shape=shape, spread=t_size)

array = levitate.models.TransducerArray(transducer_model=transducer, grid=grid)
mg = 25 * 1e-3**3 * 4 / 3 * np.pi * 9.82
def find_trap(start_pos, step_size=100e-6, path_callback=lambda pos:None, recursive=24, tol=0):
    this_pos = np.asarray(start_pos).copy()
    this_force = normalized(-levitate.optimization.gorkov_divergence(array, this_pos)(array.phases_amplitudes) - np.array([0,0,mg]))
    for _ in range(250):
        next_pos = this_pos + step_size * this_force
        next_force = normalized(-levitate.optimization.gorkov_divergence(array, next_pos)(array.phases_amplitudes) - np.array([0,0,mg]))
        if np.sum(this_force * next_force)> 0:
            path_callback(this_pos.copy())
            this_pos = next_pos
            this_force = next_force
        else:
            break

    if recursive <= 0 or step_size < tol:
        path_callback(this_pos)
        return this_pos
    else:
        return find_trap(this_pos, step_size / 2, path_callback, recursive-1)

# Calculations

In [ ]:
trap_pos = np.array([0, 0, 25e-3])
array.phases = array.focus_phases(trap_pos)
array.phases[array.num_transducers//2:] += np.pi
array.amplitudes[:] = 1

plot_pos = trap_pos

# Visualizations

## Transducers
Visualize the transducer patterns and manually set the phases/amplitudes.

In [ ]:
array.visualize_transducers(phases=lambda p:wrap_phase(p)/np.pi, transducers='first_half')
plt.show()
array.visualize_transducers(phases=lambda p:wrap_phase(p)/np.pi, transducers='last_half')
plt.show()

## Force field
Inspect the force field.

### Spatial Derivatives
Pre calculation of the spatial derivatives. Only needs to run once for a certain array geometry.

In [ ]:
point_per_mm_xy = 3
xy_min, xy_max = -10e-3, 5e-3
z_min, z_max = -2.5e-3, 2.5e-3
num_points = int((xy_max - xy_min) * point_per_mm_xy * 1e3) + 1
x_force = np.linspace(xy_min, xy_max, num_points)
y_force = np.linspace(xy_min, xy_max, num_points)
z_force = np.linspace(z_min, z_max, num_points)
progressbar_spatial_ders = IntProgress(description='Calculating:', min=0, max=len(x_force), value=0)
display(progressbar_spatial_ders)
spatial_derivatives = np.empty((len(x_force), len(y_force), len(z_force)), dtype=dict)

def work_spatial_ders(x_idx):
    spat_der_sub = np.empty((len(y_force), len(z_force)), dtype=dict)
    for  y_idx, z_idx in itertools.product(range(len(y_force)), range(len(z_force))):
        spat_der_sub[y_idx, z_idx] = array.spatial_derivatives(np.array([x_force[x_idx],y_force[y_idx],z_force[z_idx]]))
    return x_idx, spat_der_sub

def callback_spat_ders(result):
    x_idx, spat_der_sub = result
    spatial_derivatives[x_idx] = spat_der_sub
    progressbar_spatial_ders.value += 1
    if progressbar_spatial_ders.value == len(x_force):
        progressbar_spatial_ders.description = 'Done'

pool = multiprocessing.Pool(8)
for x_idx in range(len(x_force)):
    pool.apply_async(work_spatial_ders, [x_idx], callback=callback_spat_ders)
pool.close()

### Calculate and plot the field
Calculate the force field for a pre-set array state.

In [ ]:
try:
    plot_pos
except NameError:
    plot_pos = np.array([0,0,0])

Fx = np.zeros(spatial_derivatives.shape)
Fy = np.zeros(spatial_derivatives.shape)
Fz = np.zeros(spatial_derivatives.shape)
progressbar_force = IntProgress(description='Calculating', min=0, max=len(x_force), value=0)
display(progressbar_force)

def work_force(x_idx):
    Fx_sub = np.empty(spatial_derivatives[x_idx].shape)
    Fy_sub = np.empty(spatial_derivatives[x_idx].shape)
    Fz_sub = np.empty(spatial_derivatives[x_idx].shape)
    for y_idx, z_idx in itertools.product(range(len(y_force)), range(len(z_force))):
        Fx_sub[y_idx, z_idx], Fy_sub[y_idx, z_idx], Fz_sub[y_idx, z_idx] = -levitate.optimization.gorkov_divergence(
            array, None, spatial_derivatives=spatial_derivatives[x_idx, y_idx, z_idx])(array.phases_amplitudes)
    return x_idx, Fx_sub, Fy_sub, Fz_sub

def callback_force(result):
    x_idx, Fx_sub, Fy_sub, Fz_sub = result
    Fx[x_idx] = Fx_sub
    Fy[x_idx] = Fy_sub
    Fz[x_idx] = Fz_sub - mg
    progressbar_force.value += 1
    if progressbar_force.value == len(x_force):
        progressbar_force.description = 'Done'

# We can switch to multiprocessing of there are a lot of points where we want to calculate the field.
pool = multiprocessing.Pool(8)
for x_idx in range(len(spatial_derivatives)):
    pool.apply_async(work_force, [x_idx], callback=callback_force)
#     callback_force(work_force(x_idx))
pool.close()
pool.join()


Fxy = (Fx**2 + Fy**2)**0.5
Fxz = (Fx**2 + Fz**2)**0.5
F = (Fx**2 + Fy**2 + Fz**2)**0.5

x_idx = np.argmin(abs(x_force-plot_pos[0]))
y_idx = np.argmin(abs(y_force-plot_pos[1]))
z_idx = np.argmin(abs(z_force-plot_pos[2]))

plt.quiver(
    x_force*1e3, y_force*1e3,
    Fx[:, :, z_idx].T/Fxy[:, :, z_idx].T,
    Fy[:, :, z_idx].T/Fxy[:, :, z_idx].T,
    Fxy[:, :, z_idx].T*1e6, cmap='viridis',
    norm=LogNorm(1, 100),
    scale=100, scale_units='width', headlength=3, headaxislength=3, pivot='tip',
)
plt.colorbar()
plt.scatter(plot_pos[0]*1e3, plot_pos[1]*1e3, color='red')
plt.show()

if Fx.shape[2] > 1:
    plt.quiver(x_force*1e3, z_force*1e3,
        Fx[:, y_idx, :].T/Fxz[:, y_idx, :].T,
        Fz[:, y_idx, :].T/Fxz[:, y_idx, :].T,
        Fxz[:, y_idx, :].T*1e6, cmap='viridis',
        norm=LogNorm(1, 100),
        scale=100, scale_units='width', headlength=3, headaxislength=3, pivot='tip',
    )
    plt.colorbar()
    plt.scatter(plot_pos[0]*1e3, plot_pos[2]*1e3, color='red')
    plt.show()

## Pressure Field

In [ ]:
resolution_p = transducer.wavelength / 50
p_range = (130, 165)

xp_min, xp_max = np.min(array.transducer_positions[:, 0]), np.max(array.transducer_positions[:, 0])
# xp_min, xp_max = -20e-3, 20e-3
zp_min, zp_max = -array_height/2 + 2*resolution_p, array_height/2 - 2*resolution_p
# zp_min, zp_max = -5e-3, 5e-3

xz_mesh = np.meshgrid(np.arange(xp_min, xp_max, resolution_p), np.arange(zp_min, zp_max, resolution_p), indexing='ij')
pressures_xz = np.squeeze(array.calculate_pressure((xz_mesh[0], plot_pos[1]*np.ones_like(xz_mesh[0]), xz_mesh[1])))


plt.pcolormesh(1e3*xz_mesh[0], 1e3*xz_mesh[1], SPL(pressures_xz))
plt.xlabel('x')
plt.ylabel('z')
plt.clim(p_range)
plt.colorbar()
plt.scatter(plot_pos[0]*1e3, plot_pos[2]*1e3, color='red')
plt.show()